## RFM Analysis

RFM is we can say a method to analyze customer value so that we can focus on customer accordingly(We can group our customer into clusters to focus on a group having same properties similarly)

RFM stands for three dimension:

1)Recency: How recently did customer purchase(Duration since last purchase)

2)Frequency: How often did they purchase(Total number of purchases)

3)Monetary Value: How much did they spent(Total money the customer spent)

Basically RFM analysis can be done in three main steps:

1)Building the RFM feature matrix(Will contain R,F,M values corresponding to each customer)

2)Determining the cutoff point for each feature

3)Determining the RFM score for each customer and their business value

This notebook is in **Python** so the default cell type is Python. However,if we want to use different languages(Scala, SQL, and R) we can use by using adding the `%LANGUAGE` syntax at front.

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
import pandas as pd
import numpy as np

#Location and the file type
file_location = "/FileStore/tables/OnlineRetail.csv"
file_type = "csv"

#We can make the read operation(when DAG turns) much faster by defining the schema
schema = StructType([
  StructField("InvoiceNo",IntegerType(),True),
  StructField("StockCode",StringType(),True),
  StructField("Description",StringType(),True),
  StructField("Quantity",IntegerType(),True),
  StructField("InvoiceDate",StringType(),True),
  StructField("UnitPrice",FloatType(),True),
  StructField("CustomerID",IntegerType(),True),
  StructField("Country",StringType(),True)
])
df = spark.read.option("header","true") \
                         .format("csv") \
                         .schema(schema) \
                         .load("/FileStore/tables/OnlineRetail.csv")

display(df.head(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.549999952316284,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom


In [3]:
#The schema structures of the dataframe
df.printSchema()

root
-- InvoiceNo: integer (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: float (nullable = true)
-- CustomerID: integer (nullable = true)
-- Country: string (nullable = true)

In [4]:
#Columns in the data
df.columns

Out[8]: ['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [5]:
#Data Cleaning and Data Manipulation
from pyspark.sql.functions import count

#Returning the counts from all the columns to check that how many column contain how much null values(NaN values)
def my_count(df_in):
    df_in.agg( *[ count(c).alias(c) for c in df_in.columns ] ).show()
    
    
my_count(df)
#Since as we can see that our count values are not same and in CustomerID column we have some null values so we need to handle these null values

+---------+---------+-----------+--------+-----------+---------+----------+-------+
InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
 532618| 532618| 531164| 532618| 532618| 532618| 397924| 532618|
+---------+---------+-----------+--------+-----------+---------+----------+-------+

#Handling Null Values


1)The very basic way to deal with null values is to remove them only in a case if our data is much bigger so that removing the Data points with Null values doesn't affect the data much

2)Some good techniques to deal with Null values insted of removing them
As we have lot of methods to deal with null values we can replace the null values in data with some number like -999, with mean or with medain etc.

3)In this case what we can do is we can also train a model with CustomerID column as target and all other non null valued columns as our independent variables(only with non null values) and then after training replace the null values with the corresponding predicted values at that position

In [7]:
#First going with the basic method as our dataset is sufficiently large and if the model fails to generalize good on the dataset we will use other missing vaue handling techniques

df = df.dropna(how="any")
my_count(df)

#So as now we can see that the null values data points have been removed from our data

+---------+---------+-----------+--------+-----------+---------+----------+-------+
InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
 397924| 397924| 397924| 397924| 397924| 397924| 397924| 397924|
+---------+---------+-----------+--------+-----------+---------+----------+-------+

In [8]:
#Converting InvoiceDate coulmn to UTC time stamp format(New column NewInvoiceDate Created)

from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, lit, datediff, col
timeFormat = "MM/dd/yy HH:mm"
df = df.withColumn("NewInvoiceDate",to_utc_timestamp(unix_timestamp(col("InvoiceDate"),timeFormat).cast("timestamp"),"UTC"))
display(df.head(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,NewInvoiceDate
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.549999952316284,17850,United Kingdom,2010-12-01T08:26:00.000+0000
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01T08:26:00.000+0000
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000


In [9]:
#Calculating the total price
#For Calculating the monetary value we will be requiring the total amount that the customer has spent(so we need to get the price spent  by customer and which is equal to Quantity*unit_price_of_quantity)
from pyspark.sql.functions import round
df = df.withColumn("TotalPrice",round(df.Quantity*df.UnitPrice,2))
display(df.head(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,NewInvoiceDate,TotalPrice
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.549999952316284,17850,United Kingdom,2010-12-01T08:26:00.000+0000,15.300000190734863
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000,20.34000015258789
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01T08:26:00.000+0000,22.0
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000,20.34000015258789
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000,20.34000015258789


In [10]:
#Calculating the time difference

from pyspark.sql.functions import mean,min,max,sum,datediff,to_date
date_max = df.select(max("NewInvoiceDate")).toPandas()
current = to_utc_timestamp(unix_timestamp(lit(str(date_max.iloc[0][0])),"yy-MM-dd HH:mm").cast("timestamp"),"UTC")

#Calculating the Duration(Duration is another important attribute for RFM analysis which tell how often did customer purchase)(From how much time he hasn't purchased)

df = df.withColumn("Duration",datediff(lit(current),"NewInvoiceDate"))
display(df.head(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,NewInvoiceDate,TotalPrice,Duration
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.549999952316284,17850,United Kingdom,2010-12-01T08:26:00.000+0000,15.300000190734863,373
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000,20.34000015258789,373
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01T08:26:00.000+0000,22.0,373
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000,20.34000015258789,373
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390000104904175,17850,United Kingdom,2010-12-01T08:26:00.000+0000,20.34000015258789,373


###In this step we have build our RFM feature matrix and hence completed our first step of RFM analysis

In [12]:
#Building Recency,Frequency and Monetary attribute corresponding to the customers ID(Customers)

recency = df.groupBy("CustomerID").agg(min("Duration").alias("Recency"))
frequency = df.groupBy("CustomerID","InvoiceNo").count().groupBy("CustomerID").agg(count("*").alias("Frequency"))
monetary = df.groupBy("CustomerID").agg(round(sum("TotalPrice"), 2).alias("Monetary"))
rfm = recency.join(frequency,"CustomerID",how ="inner").join(monetary,"CustomerID",how ="inner")

display(rfm.head(5))

CustomerID,Recency,Frequency,Monetary
15727,16,7,5178.96
16574,71,1,451.44
17420,50,3,598.83
13285,23,4,2709.12
15619,10,1,336.4


Now our next step will be determination of cutting points for each feature

RFM Segamentation
Determine the cutting points for each attribute.Cutting points can be mainly inferenced from business point of view.

In [15]:
def describe_pd(df_in, columns, deciles=False):
    if deciles:
        percentiles = np.array(range(0, 110, 10))
    else:
        percentiles = [25, 50, 75]
    percs = np.transpose([np.percentile(df_in.select(x).collect(),percentiles) for x in columns])
    percs = pd.DataFrame(percs,columns=columns)
    percs["summary"] = [str(p) + "%"for p in percentiles]
    spark_describe = df_in.describe().toPandas()
    new_df = pd.concat([spark_describe, percs],ignore_index=True)
    new_df = new_df.round(2)
    return new_df[["summary"] + columns]

In [16]:
cols = ["Recency","Frequency","Monetary"]
describe_pd(rfm,cols,1)

,summary,Recency,Frequency,Monetary
0,count,4339,4339,4339
1,mean,92.0414842129523,4.271952062687255,2053.793012214796
2,stddev,100.00775734416372,7.70549277131482,8988.248319510309
3,min,0,1,0.0
4,max,373,210,280206.02
5,0%,0,1,0
6,10%,5,1,156.566
7,20%,13.2,1,250.106
8,30%,22,1,351.658
9,40%,32,2,489.724


In [17]:
#Use obove function describe_pd or either use this below piece of code for short statistical inference
cols = ["Recency","Frequency","Monetary"]
rfm.select(cols).describe().show()

+-------+------------------+-----------------+-----------------+
summary| Recency| Frequency| Monetary|
+-------+------------------+-----------------+-----------------+
 count| 4339| 4339| 4339|
 mean| 92.0414842129523|4.271952062687255|2053.793012214796|
 stddev|100.00775734416372| 7.70549277131482|8988.248319510309|
 min| 0| 1| 0.0|
 max| 373| 210| 280206.02|
+-------+------------------+-----------------+-----------------+

In [18]:
#Using the quantile for defining the R,F,M values between 1 and 4
#According to the magnitudes we have assigned values between 1 to 4 to the attributes

def RScore(x):
  #Smaller value of x(Recency) tells us that the particular customer has done some activity(like buying something or using some product) recently and contrary larger the value of x will give some inference that customer wasn't involved in activity from a long time
  if x <= 16:
    return 1
  elif x<= 50:
    return 2
  elif x<= 143:
    return 3
  else:
    return 4

def FScore(x):
  #Smaller the value of x(Frequency) tell that the customer is not involved in activities frequently and for customer with high value of x denotes that customer is involved in Frequent activities
  if x <= 1:
    return 4
  elif x <= 3:
    return 3
  elif x <= 5:
    return 2
  else:
    return 1

def MScore(x):
  #Smaller the value of x(Monetary value) tells us that customer activities cost is not much(has not spent much money on buying some product etc) and contrary higher value of x denotes that customer has spent a lot of money on activities
  if x <= 293:
    return 4
  elif x <= 648:
    return 3
  elif x <= 1611:
    return 2
  else:
    return 1

#A customer can have any of the permutation of these values corresponding to their activities
  

#For each and every value of R,F,M we will pass them through the lambda function in corresponding R_udf,F_udf,M_udf
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType

R_udf = udf(lambda x: RScore(x), StringType())
F_udf = udf(lambda x: FScore(x), StringType())
M_udf = udf(lambda x: MScore(x), StringType())

#RFM segamentation
from pyspark.sql.functions import concat

rfm_seg=rfm.withColumn("r_seg", R_udf("Recency"))
rfm_seg=rfm_seg.withColumn("f_seg", F_udf("Frequency"))
rfm_seg=rfm_seg.withColumn("m_seg", M_udf("Monetary"))
display(rfm_seg.head(5))

CustomerID,Recency,Frequency,Monetary,r_seg,f_seg,m_seg
15727,16,7,5178.96,1,1,1
16574,71,1,451.44,3,4,3
17420,50,3,598.83,2,3,3
13285,23,4,2709.12,2,2,1
15619,10,1,336.4,1,4,3


##Next will be determination of RFM score and corresponding business value

In [20]:
col_list=["r_seg","f_seg","m_seg"]

#RFM score is nothing but the concatenated R,F,M values
rfm_seg=rfm_seg.withColumn("RFMScore",concat(*col_list))
display(rfm_seg.sort("RFMScore").head(5))

CustomerID,Recency,Frequency,Monetary,r_seg,f_seg,m_seg,RFMScore
17809,16,12,5411.91,1,1,1,111
17677,1,30,16363.23,1,1,1,111
16255,16,8,2619.91,1,1,1,111
15382,14,6,5955.71,1,1,1,111
12757,15,13,7857.1,1,1,1,111


In [21]:
#Statistical summary for each RFM score(Mapping of RFM score against average R,F,M values)
display(rfm_seg.groupBy("RFMScore").agg({"Recency":"mean","Frequency":"mean","Monetary":"mean"} ).sort(["RFMScore"]).head(5))

RFMScore,avg(Recency),avg(Monetary),avg(Frequency)
111,6.175398633257403,9644.012460136679,16.439635535307517
112,7.892857142857143,1243.5928571428572,7.285714285714286
113,9.333333333333334,443.08666666666664,6.0
114,11.0,201.12,7.0
121,7.5131578947368425,2638.8373684210515,4.5131578947368425


RFM Value Inference

Now the main conclusion is that what and how these RFM values are analyzed

Based on RFM values we can have much possible categories of customers some categories can be of overlapping nature with other category(means can be included within one category broadly)

1)The First case is customer with R,F,M values 1,1,1: The customers with these values are the one's who does activities(like buying,using product) more recently more oftenly and spends the most

2)The Second case is customer with R,F,M values 4,4,4: The Customer with these R,F,M values are the one's who does activities less recently and less oftenly and they spents little

The above two were the extreme cases of customers out of which some can be said as best customer and contrary from organization business point of view.

Also leaving the extreme cases we can also have some other variation in customers:

1)There are also customers that appear less frequently but spends a lot with high monetary value

2)Some customers appear more frequently and oftenly but spends a little they have high Recency and Frequency value

3)We also have customers who appears frequntly they can be termed as loyal customers

4)There can be customer who have not purchased from a long time but appear but comes and make a large monetary value transaction

RFM analysis is one of the key technique in gaining insights about the customers, like what kind of customers does we have and how can we cluster them in one category so that we can focus on categories with large number of customers in each category and provide some king of offers to them before their churning